Install the auto_echem package. You need git installed onto your computer (https://git-scm.com/download/win)

In [ ]:
pip install "git+https://github.com/lfo96/auto_echem.git"  

This needs to be installed only once. Can be removed from the notebook thereafter.  
I reguarly update the functions, so if you want to have the latest version on here either keep your local respitory udpated or run the pip update from time to time.

In [ ]:
pip install -U "git+https://github.com/lfo96/auto_echem.git

Create notebook in a directionary where you also want to store your plots. I usually create a new notebook for every new set of measurements I want to evaluate.  
Import the functions from auto_echem into your notebook:

In [ ]:
# Those are the functions you need 
from auto_echem.GCPL_functions import *
from auto_echem.general_functions import *
from auto_echem.three_el_functions import *
from auto_echem.auto import *
from auto_echem.impedance_functions import *
from auto_echem.TGA_DSC_functions import *
from auto_echem.XPS_functions import *

Next I specify where my biologic data is stored. Conveniently you can map the mp_pastagroup network drive where the data is automatically updated in real time (or whenever backup360 creates its backups...)  
Always specify the lab setting file created by ec-lab. It has the .mps format. You can use the test files in "auto_echem\testfiles\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1.mps" to try and run this notebook.

note: use r' before your pathway to make sure python can read it properly

In [ ]:
test_file_p = r"packagelocation\auto_echem\testfiles\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1.mps"

From here your analysis is dependent on your measurement file.  
auto_echem functions can read in and process various measurements file generated by EC-lab (and BT-lab), but also TGA-DSC, XRD, XPS.
\
\
The most generic analysis uses the "auto" function. The function reads in the .mps file, extract the specified measurements and subsequently look for through the folder of the setting file for the respesctive .mpr files. Each .mpr file is read in and evaluated respectivly.


In [ ]:
test_file = auto(test_file_p, circ = ['RpC'])

Usually I let the auto function run over my measurement and continue from there on. The auto function returns a dictionary (dict) with meta data, all the raw data (dict['data']), and the evaluated raw data (dict['eva']).

In this example, we analyzed a measurement of a coin cell, starting with a resting period measuring the OCV, PEIS, and looping 23 times. Subsequently the GCPL is measured. 

The auto function has detected and analyzed OCV, PEIS and GCPL in the measurement folder.
The dictionary contains amongst other, the following information: 

In [ ]:

print('Your active material mass is (mg):',test_file['active material mass'])
print('Your electrode surface area is (cm2):',test_file['electrode surface area'])
print('Your test protocol is:',test_file['protocol'])

Your raw data is saved into the dictionary and can be accessed by dict['data']. Therein each measurement gets its own entry with a number indicating the chronology of the measurement. For example:

In [ ]:
for meas_data in test_file['data']:
    print(meas_data)

Similiarily, the raw data (stored in ['data']) is processed, evaluated, and stored in dict['eva'].

In [ ]:
for eva_data in test_file['eva']:
    print(eva_data)

Since there is not much to analzye for OCV measurement, there is no entry in the 'eva' section.  
The "1 PEIS" contains various information including for example each parameter order by cycle.


In [ ]:
test_file['eva']['1 PEIS']['Nyquist parameter']